In [1]:
import nltk
nltk.download('stopwords')
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arasu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import json
import pandas as pd
data_file = open("yelp_academic_dataset_review.json",'r', encoding='utf-8')
data = []
p=0
for line in data_file:
    if(p<100000):
        p+=1
        continue
    if(p>200000):
        break
    data.append(json.loads(line))
    p+=1
review_df = pd.DataFrame(data)
data_file.close()

In [3]:
review_df.iloc[1,:]

review_id                                 NFckLwN9S4UgFlfo0bPRxA
user_id                                   HwcjEqA1V3bKtb07JKGIKQ
business_id                               nqKL5PbJbwwoCK_Xon31kA
stars                                                        4.0
useful                                                         0
funny                                                          0
cool                                                           0
text           I went here for a friend's birthday with a lar...
date                                         2007-06-15 16:13:15
Name: 1, dtype: object

In [4]:
lipos=[(i,1) for i in review_df[(review_df['stars']==4 )|(review_df['stars']==5)]['text']]
linos=[(i,0) for i in review_df[(review_df['stars']==1 )|(review_df['stars']==2)]['text']]
li0=[(i,0.5) for i in review_df[(review_df['stars']==3 )]['text']]

In [5]:
import numpy as np
arr=lipos+linos
np.random.seed(10)
np.random.shuffle(arr)
len(arr)

87849

In [6]:
def process_tweet(tweet):
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)

    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                   reduce_len=True)

    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)
    stopwords_english = stopwords.words('english') 
    tweets_clean = []

    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [] 

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list

    #print('stemmed words:')
    #print(tweets_stem)
    return tweets_stem
import random
x=random.randint(1,(len(arr)))
process_tweet(arr[x][0])

["i'm",
 'sure',
 'fuss',
 '4',
 'star',
 'basic',
 'hotel',
 'restaur',
 'overpr',
 'regular',
 'food',
 'get',
 'street',
 'restaur',
 'burlington',
 'mall',
 'decor',
 'nice',
 'kobe',
 'burger',
 'tast',
 'like',
 'burger',
 'say',
 'onion',
 'ring',
 'great',
 'drink',
 'eh',
 'servic',
 'great',
 'price',
 'menu',
 'weird',
 "they'r",
 'wear',
 'jean',
 'speak',
 'custom',
 'expect',
 'spoken',
 'burger',
 'king',
 'probabl',
 'come',
 'back']

In [7]:
# Train test split 
train=arr[:(len(arr)*80//100)]
test=arr[(len(arr)*80//100):]

In [9]:
def freq(arr): # Creating frequency dictionary
    freqs=dict()
    k=0
    for i in arr:
        k+=1
        if k%10000==0:
            print(k//10000)
        for j in process_tweet(i[0]):
            if (j,i[1]) in freqs:
                freqs[(j,i[1])]=freqs[(j,i[1])]+1
            else:
                freqs[(j,i[1])]=1
    return freqs
freqs=freq(train)

1
2
3
4
5
6
7


In [10]:
tweets=[i[0] for i in train]
y=[i[1] for i in train]
# Exract features for training set using frequency dictionary

def extract_features(tweet, freqs):
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    for word in word_l:
        if (word,1) in freqs:
            x[0,1] += freqs[(word,1)]
        if (word,0) in freqs:
            x[0,2] += freqs[(word,0)]
    norm = np. linalg. norm(x)
    x=x/norm
    assert(x.shape == (1, 3))
    return x
xfeats=np.empty((1,3))
p=0
for i in tweets:
    if(p%10000==0):
        print(p//10000)
    p+=1
    xfeats=np.append(xfeats,extract_features(i, freqs), axis=0)

0
1
2
3
4
5
6
7


In [11]:
y1=np.array(y).reshape(-1,1)
xfeats1=np.delete(xfeats,0,axis=0)

In [12]:
def sigmoid(z):
    return 1/(1+np.exp(-1*z))
def cost_function(theta,X,y):
    z=np.dot(X,theta)
    #print(z)
    h=sigmoid(z)
    #print(h)
    return (-1/len(X))*np.dot(np.transpose(np.log(h)),y)+(-1/len(X))*np.dot(np.transpose(np.log(1-h)),(1-y))
def gradient(theta,X,y):
    z=np.dot(X,theta)
    #print(z.shape)
    h=sigmoid(z)
    #print((h).shape)
    #print(y.shape)
    return (np.dot(np.transpose(X),h-y)/len(X))

m=xfeats.shape[0]
cost_the=list()
theta=np.zeros((3, 1))
alpha=1e-1
num_iters=320000
# Training Logistic Regression model
for i in range(num_iters):
    
    J=cost_function(theta,xfeats1,y1)
    
    theta=theta-alpha*gradient(theta,xfeats1,y1)

    if(i%10000==0):
        print(J,i)
        cost_the.append((J,theta))

[[0.69314718]] 0
[[0.38701409]] 10000
[[0.35170652]] 20000
[[0.33888552]] 30000
[[0.33308305]] 40000
[[0.33012871]] 50000
[[0.32851276]] 60000
[[0.32758609]] 70000
[[0.32703691]] 80000
[[0.32670361]] 90000
[[0.32649773]] 100000
[[0.32636884]] 110000
[[0.32628732]] 120000
[[0.32623534]] 130000
[[0.32620198]] 140000
[[0.32618047]] 150000
[[0.32616653]] 160000
[[0.32615746]] 170000
[[0.32615155]] 180000
[[0.32614768]] 190000
[[0.32614514]] 200000
[[0.32614346]] 210000
[[0.32614235]] 220000
[[0.32614161]] 230000
[[0.32614111]] 240000
[[0.32614076]] 250000
[[0.32614053]] 260000
[[0.32614036]] 270000
[[0.32614023]] 280000
[[0.32614013]] 290000
[[0.32614006]] 300000
[[0.32613999]] 310000


In [13]:
#Extracting features for test set
tweets1=np.array([i[0] for i in test])
ytest=np.array([i[1] for i in test])
xfeats2=np.empty((1,3))
p=0
for i in tweets1:
    if(p%10000==0):
        print(p//10000)
    p+=1 
    xfeats2=np.append(xfeats2,extract_features(i, freqs), axis=0) 

0
1


In [14]:
theta

array([[  0.21248413],
       [ 12.90096677],
       [-28.69168818]])

In [15]:
x_test=np.delete(xfeats2,0,axis=0)
ypreds=[0 if i<0.5 else 1 for i in sigmoid(np.dot(x_test,theta))]
sum([1 if(ytest[i]==ypreds[i]) else 0 for i in range(len(ypreds))])/len(ypreds) # Accuracy score on test data

0.8578258394991463